In [15]:
!pip install yfinance
!pip install yahooquery
from yahooquery import Screener
import yfinance as yf   
import pandas as pd
import numpy as np
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,LSTM
from tensorflow.keras.layers import Dropout
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import NearestCentroid
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
import tensorflow as tf
from sklearn.tree import DecisionTreeClassifier
from sys import getsizeof

In [3]:
def read_syms_from_txt():  
  with open("syms.txt","r") as f:
    lines = f.readlines()
    f.close()
  lst = []
  for line in lines:
    line = line.strip()
    lst.append(line)
  symbols = lst
  return symbols
def get_crypto_syms():
  #'all_cryptocurrencies_au','all_cryptocurrencies_ca','all_cryptocurrencies_eu','all_cryptocurrencies_gb','all_cryptocurrencies_in',
   screens = [
 'all_cryptocurrencies_us','all_cryptocurrencies_au','all_cryptocurrencies_ca','all_cryptocurrencies_eu','all_cryptocurrencies_gb','all_cryptocurrencies_in']
   s = Screener()
   symbols = []
   for i in screens:
      data = s.get_screeners(i, count=250)
      dicts = data[i]['quotes']
      syms = [d['symbol'] for d in dicts]
      for sym in syms:
        symbols.append(sym)
   #print(len(symbols))
   #pieces = 15
   # new_arrays = np.array_split(symbols, pieces)
   return symbols
def spliting(data):
  X = data.drop(["sugg"],axis=1)
  y = data["sugg"]
  xTrain, xTest, yTrain, yTest = train_test_split(X, y, test_size=0.1,random_state=99)
  print(xTrain.shape,end=" ")
  print(yTrain.shape)
  print(xTest.shape,end=" ")
  print(yTest.shape)
  return xTrain, xTest, yTrain, yTest
def extract_data(df):
    rows = []
    for each in range(6,df.shape[0]-1):
        sugg = 0
        if df[each+1][3] > df[each][3]:
          sugg = 1
        row = [
                df[each-6][3],
                df[each-5][3],
                df[each-4][3],
                df[each-3][3],
                df[each-2][3],
                df[each-1][3],
                df[each][3], 
                sugg
        ]
        rows.append(row)
    return rows
def row_scaler(df):
  scaler = MinMaxScaler(feature_range=(0,1))
  last_column = df.iloc[: , -1]
  df = df.drop(columns=df.columns[-1], axis=1)
  scaled = pd.DataFrame(scaler.fit_transform(df.T).T,dtype=object)
  scaled["sugg"] = last_column
  return scaled
def stick_dfs(dfs):
  dataframe = dfs[0]
  for i in range(1,len(dfs)):
     dataframe = pd.concat([dataframe,dfs[i]], ignore_index = True)
  dataframe = dataframe.dropna()
  dataframe = dataframe.astype(float)
  return dataframe
def process(dfs): 
   fixed_dfs = []
   for df in dfs:
      df = np.array(df)
      df = extract_data(df)
      df = pd.DataFrame(df)
      df = row_scaler(df)
      fixed_dfs.append(df)
   df = stick_dfs(fixed_dfs)
   return df      

In [29]:
#symbols = ["MSFT","AAPL","GOOG","TSLA","AMZN"]
#symbols = ["BTC-USD","LTC-USD","TRX-USD","XRP-USD","ETH-USD","BNB-USD","DASH-USD","VET-USD","LINK-USD","ADA-USD","DOT-USD","SOL-USD","BCH-USD","FTT-USD","FIL-USD","XMR-USD"]
#symbols = ["AAPL","MSFT","TSLA","GOOG"]
#symbols = ["BTC-USD","ETH-USD"]
#symbols = ["BTC-USD"]
#df = pd.read_parquet("/content/drive/MyDrive/Colab Files/250_crypto_1d_scaled_yf.parquet")

symbols = get_crypto_syms()
dfs = []
for symbol in symbols:
           data = yf.download(symbol,period="MAX",interval="1d",progress=False)
           if data.empty :
             print("Passing...")
           else:
               dfs.append(data)
ndfs = dfs
dfs = []
for df in ndfs:
        if df.shape[0] > 7:
          dfs.append(df)
data = process(dfs)
data.to_csv("1d_6pc_10mms.csv")
xTrain, xTest, yTrain, yTest = spliting(data)
data


1 Failed download:
- XNO-AUD: No data found for this date range, symbol may be delisted
Passing...

1 Failed download:
- OXEN-AUD: No data found for this date range, symbol may be delisted
Passing...

1 Failed download:
- VAL-AUD: No data found for this date range, symbol may be delisted
Passing...

1 Failed download:
- ICP-CAD: No data found for this date range, symbol may be delisted
Passing...

1 Failed download:
- XNO-CAD: No data found for this date range, symbol may be delisted
Passing...

1 Failed download:
- OXEN-CAD: No data found for this date range, symbol may be delisted
Passing...

1 Failed download:
- VAL-CAD: No data found for this date range, symbol may be delisted
Passing...

1 Failed download:
- ICP-EUR: No data found for this date range, symbol may be delisted
Passing...

1 Failed download:
- XNO-EUR: No data found for this date range, symbol may be delisted
Passing...

1 Failed download:
- OXEN-EUR: No data found for this date range, symbol may be delisted
Passing.

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,sugg
0,1.000000,0.880182,0.873523,0.418129,0.413395,0.001728,0.000000,0.199899,0.188394,0.058266,0.062185,0.105055,0.104213,0.577573,0.0
1,1.000000,0.478669,0.473250,0.001978,0.000000,0.228842,0.215672,0.066702,0.071189,0.120266,0.119302,0.661199,0.660556,0.458810,0.0
2,0.715745,0.002991,0.000000,0.346101,0.326183,0.100881,0.107666,0.181891,0.180432,1.000000,0.999028,0.693905,0.692714,0.411037,0.0
3,0.000000,0.346101,0.326183,0.100881,0.107666,0.181891,0.180432,1.000000,0.999028,0.693905,0.692714,0.411037,0.407510,0.237171,0.0
4,0.250581,0.000000,0.007546,0.090100,0.088477,1.000000,0.998919,0.659562,0.658237,0.344955,0.341033,0.151582,0.128077,0.018907,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1766921,0.000000,0.007940,0.006025,0.286972,0.286972,0.277056,0.277024,0.162749,0.162749,0.530168,0.536346,0.665823,0.667522,1.000000,1.0
1766922,0.000000,0.237021,0.237021,0.228654,0.228628,0.132220,0.132220,0.442191,0.447404,0.556636,0.558069,0.838563,0.836661,1.000000,1.0
1766923,0.118140,0.108709,0.108679,0.000000,0.000000,0.349424,0.355300,0.478435,0.480051,0.796245,0.794100,0.978229,0.976466,1.000000,0.0
1766924,0.108679,0.000000,0.000000,0.349424,0.355300,0.478435,0.480051,0.796245,0.794100,0.978229,0.976466,1.000000,0.995446,0.719797,0.0


In [ ]:
data = process(dfs)
data.to_csv("1d_6pc_10mms.csv")
xTrain, xTest, yTrain, yTest = spliting(data)
data

In [39]:
model = Sequential()
adam = tf.keras.optimizers.Adam(
    learning_rate=3e-4,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-07,
    amsgrad=True,
    name="Adam")

model.add(Dense(1000, activation='relu', input_shape=(xTrain.shape[1],)))
model.add(Dense(750, activation='relu'))
model.add(Dense(750, activation='relu'))
model.add(Dense(250, activation='relu'))

model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer=adam, loss='binary_crossentropy',metrics=['accuracy'])
model.summary()

Model: "sequential_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_70 (Dense)            (None, 1000)              15000     
                                                                 
 dense_71 (Dense)            (None, 750)               750750    
                                                                 
 dense_72 (Dense)            (None, 750)               563250    
                                                                 
 dense_73 (Dense)            (None, 250)               187750    
                                                                 
 dense_74 (Dense)            (None, 1)                 251       
                                                                 
Total params: 1,517,001
Trainable params: 1,517,001
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(xTrain,yTrain,epochs=100,batch_size=4000,validation_data=(xTest,yTest))
model.evaluate(xTest,yTest)
model.save(f"{model.evaluate(xTest,yTest)}.h5")

Epoch 1/100
398/398 [==============================] - 14s 33ms/step - loss: 0.6867 - accuracy: 0.5416 - val_loss: 0.6849 - val_accuracy: 0.5480
Epoch 2/100
398/398 [==============================] - 12s 30ms/step - loss: 0.6819 - accuracy: 0.5552 - val_loss: 0.6801 - val_accuracy: 0.5606
Epoch 3/100
398/398 [==============================] - 13s 32ms/step - loss: 0.6779 - accuracy: 0.5651 - val_loss: 0.6768 - val_accuracy: 0.5667
Epoch 4/100
398/398 [==============================] - 13s 34ms/step - loss: 0.6742 - accuracy: 0.5720 - val_loss: 0.6728 - val_accuracy: 0.5754
Epoch 5/100
398/398 [==============================] - 13s 32ms/step - loss: 0.6708 - accuracy: 0.5784 - val_loss: 0.6704 - val_accuracy: 0.5786
Epoch 6/100
398/398 [==============================] - 13s 33ms/step - loss: 0.6670 - accuracy: 0.5847 - val_loss: 0.6682 - val_accuracy: 0.5837
Epoch 7/100
398/398 [==============================] - 13s 33ms/step - loss: 0.6634 - accuracy: 0.5903 - val_loss: 0.6642 - val_ac

In [ ]:
clf = RandomForestClassifier(n_estimators=100)
clf.fit(xTrain,yTrain)
print(f"Random Forest :  {clf.score(xTest,yTest)}")
logisticRegr = LogisticRegression()
logisticRegr.fit(xTrain, yTrain)
print(f"Logistic Regression: {logisticRegr.score(xTest, yTest)}")
nc = NearestCentroid()
nc.fit(xTrain,yTrain)
print(f"Nearest Centroid: {nc.score(xTest,yTest)}")
gnb = GaussianNB()
gnb.fit(xTrain, yTrain)
print(f"GaussianNB: {gnb.score(xTest,yTest)}")
clf2 = KNeighborsClassifier(n_neighbors=2)
clf2.fit(xTrain, yTrain)
print(f"K-Neighbors: {clf2.score(xTest,yTest)}")
tree = DecisionTreeClassifier()
tree.fit(xTrain, yTrain)
print(f"Decision Tree: {tree.score(xTest,yTest)}")
gb = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,
   max_depth=1, random_state=0).fit(xTrain, yTrain)
gb.fit(xTrain,yTrain)
print(f"Gradient Boost: {gb.score(xTest, yTest)}")
svm = SVC()
svm.fit(xTrain,yTrain)
print(f"SVM: {svm.score(xTest,yTest)}")

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive/")

address = "/content/gdrive/MyDrive/Colab Files/1d_6pc_10mms.csv"
data = pd.read_csv(address)
xTrain, xTest, yTrain, yTest = spliting(data)
data